In [2]:
import pandas as pd
import matplotlib
from matplotlib import pylab as plt
import numpy as np

In [3]:
df = pd.read_csv('../Data/2005-2010_Graduation_Outcomes_-_School_Level.csv')
df = df[df['Demographic'] == 'Total Cohort']
df['Demographic'].value_counts()

Total Cohort    2493
Name: Demographic, dtype: int64

In [4]:
# creates Boro feature from DBN
df['BORO'] = [df.loc[i,'DBN'][2] for i in df.index]
df['BORO'].isnull().sum()

0

In [5]:
#Median Household Income used to merge with df
income_list = [['01', 42065], ['02', 92628], ['03', 110371], ['04', 35344], ['05', 36697], ['06', 41693], 
               ['07', 18237], ['08', 44490], ['09', 27243], ['10', 24590], ['11', 52782], ['12', 24590], ['13', 67081], 
               ['14', 39966], ['15', 67081], ['16', 40424], ['17', 48788], ['18', 68888], ['19', 40519], ['20', 63782], 
               ['21', 33209], ['22', 53997], ['23', 29125], ['24', 53907], ['25', 64493], ['26', 86854], ['27', 60748], 
               ['28', 60901], ['29', 88630], ['30', 54001], ['31', 78082], ['32', 33485]]
df_income = pd.DataFrame(income_list, columns = ['District', 'Income'])

In [6]:
# DON'T RUN TWICE!!!!
# if run twice 2006 column will be deleted
# creates 2006 columns
#df = df[(df.Cohort != '2006')]
#df.loc[ : , 'Cohort'] = df.loc[ : , 'Cohort'].replace('2006 Aug', '2006')

In [7]:
df.groupby('Cohort').Cohort.agg('count')

Cohort
2001    262
2002    318
2003    346
2004    367
2005    390
2006    405
Name: Cohort, dtype: int64

In [8]:
# creates district column the merges with income data for each district
df['District'] = [df.loc[i,'DBN'][0:2] for i in df.index]
df = df.merge(df_income, on='District')

In [9]:
# creates target variable of next years graduation rate
df.loc[ : ,'Merge Cohort'] = df['Cohort'].astype(int) + 1
df.loc[ : ,'Cohort'] = df['Cohort'].astype(int)
df_merge = df[['DBN', 'Cohort', 'Total Grads - % of cohort']]
df_merge = df_merge.rename(columns={"Cohort": "Cohort 2", 'Total Grads - % of cohort': 'y'})

df_master = df.merge(df_merge, 'left', left_on = ['DBN', 'Merge Cohort'], right_on = ['DBN','Cohort 2'])
df_master = df_master.drop(['Merge Cohort', 'Cohort 2'], axis=1)
df_master.loc[ : ,'Cohort'] = df_master['Cohort'].astype(str)
df = df_master

In [10]:
df.to_csv('../Data/cleaned_data2.csv')